# 데이터 타입, 함수, 연산자

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 숫자형과 함수

- 빅쿼리는 숫자, 문자, 시간, 지리 데이터, 정형, 반정형 데이터등 다양한 데이터 타입을 지원
    - int64
        : 유일한 정수형 실수이면 float64, 불리언이면 bool
    - numeric
        : 정확한 계산이 필요할 때 적합
    - string
        : 가변 길이의 유니코드 문자열 표현
    - timestamp
        : 시간의 절대 시점
    - datetime
        : 달력상의 날짜와 시간
    - geography
        : 지구 표면의 점, 선, 폴리곤
    - struct, array
        : 구조체와 배열

In [2]:
query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
)
select *,(oneways / numrides) as frac_oneway
from example
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.04rows/s]


,day,numrides,oneways,frac_oneway
0,sat,1451,1018,0.701585
1,sun,2376,936,0.393939


- 함수유형
    - 스칼라(scalar) 함수 : 하나 이상의 입력 파라미터에서 작동해 단일 값을 반환하는 함수
        - ex) round( ), substr( )
    - 집계(aggregate) : 계산을 수행하고 단일 값을 반환하는 함수 , 종종 group by와 함께 사용  
        - ex) max( ), sum( ), count( ), avg( )
    - 분석(analytics)함수 : 값 모음에서 작동하지만 모음의 각 값에 대한 결과를 반환
        - ex) row_number( ), rank( )
    - 테이블 반환 함수 : from절에서 사용할 수 있는 결과 집합을 반환
        - ex) 배열에서 unnest를 호출한 다음 선택
    - 사용자 정의 함수 : 사용자가 직접 구현한 함수


## 수학 함수

In [3]:
# round함수를 사용해 결과를 반올림하는 쿼리
query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
)
select *,round((oneways / numrides),2) as frac_oneway
from example
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.09rows/s]


,day,numrides,oneways,frac_oneway
0,sat,1451,1018,0.70
1,sun,2376,936,0.39


## 표준 규격 부동소수점 분할

In [5]:
# IEEE_DIVIDE를 사용한 쿼리
query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
    union all select 'wnd',0,0
)

select *, round(ieee_divide(oneways,numrides),2) as frac_oneway
from example
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.98rows/s]


,day,numrides,oneways,frac_oneway
0,sat,1451,1018,0.70
1,sun,2376,936,0.39
2,wnd,0,0,NaN


## safe 함수

In [6]:
query ='''
SELECT LOG(10,3), SAFE.lOG(10,-3)
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.03rows/s]


,f0_,f1_
0,2.095903,NaN


## numeric을 사용한 정밀 소수 계산

In [15]:
#numeric을 사용한 소수점 계산
query = '''
with example as(
    select numeric '1.23' as payment
    union all select numeric'7.89'
    union all select numeric'12.43'
)

select
    sum(payment) as total_paid,
    avg(payment) as average_paid
from example 
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df
# 평균은 반복되는 소수점이므로 numeric 타입으로도 정확한 계산이 불가능

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.92rows/s]


,total_paid,average_paid
0,21.55,7.183333333


# 불(bool) 다루기

## 논리연산

In [17]:
query='''
with example as (
    select null as is_vowel, null as letter, -1 as position
    union all select true, 'a',1
    union all select false, 'b',2
    union all select false, 'c',3
)

select *
from example 
where is_vowel != false'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.98rows/s]


,is_vowel,letter,position
0,True,a,1


In [18]:
query='''
with example as (
    select null as is_vowel, null as letter, -1 as position
    union all select true, 'a',1
    union all select false, 'b',2
    union all select false, 'c',3
)

select *
from example 
where is_vowel is not false'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

#비교연산자는 null과 비교하면 null을 반환하지않지만 is 연산자는 null값 또한 반환

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.93rows/s]


,is_vowel,letter,position
0,None,None,-1
1,True,a,1


## 조건식

In [19]:
query = '''
with catalog as(
    select 30.0 as costprice, 0.15 as markup, 0.1 as taxrate
    union all select null, 0.21, 0.15
    union all select 30.0, null, 0.09
    union all select 30.0, 0.30, null 
    union all select 30.03, null, null
)

select *, round( costprice * if(markup is null, 1.05, 1+markup)* if(taxrate is null, 1.10,1+taxrate),2) as salesprice
from catalog 
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df


Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.45rows/s]


,costprice,markup,taxrate,salesprice
0,30.00,0.15,0.10,37.95
1,NaN,0.21,0.15,NaN
2,30.00,NaN,0.09,34.34
3,30.00,0.30,NaN,42.90
4,30.03,NaN,NaN,34.68
